In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [86]:
df = pd.read_csv("./2.csv", delimiter=";")
df.head()

,ENROLLMENT,TUITION PAYMENT MARCH 2022,TUITION PAYMENT MARCH 2023,GENDER,TYPE OF EDUCATIONAL INSTITUTION,EDUCATIONAL INSTITUTION,INSTITUTION STATUS,DEPARTMENT,PROVINCE,DISTRICT,...,CAMPUS,FACULTY,PROGRAM/MAJOR,SHIFT/SCHEDULE,BENEFIT DISCOUNTS,STUDY MODE,AGE RANGE OF ENROLLED STUDENT,DISABILITY,NUMBER OF ENROLLED COURSES,AT-RISK COURSE
0,Nuevo,0,0,M,INSTITUTO,IDAT,PRIVADA,LIMA,LIMA,BRENA,...,UTP Lima Centro,Fac. Ing. Sist. Y Elect.,ING. DE REDES Y COMUNICACIONES,MIXTO,SIN BENEFICIO,Presencial,5. >=30,No,0,0
1,Nuevo,1,0,M,COLEGIO,COLEGIO SISE,PRIVADA,LIMA,LIMA,VILLA MARIA DEL TRIUNFO,...,UTP Lima Centro,Fac. Ing. Sist. Y Elect.,ING. DE SISTEMAS,NOCHE,SIN BENEFICIO,Presencial,4. 24-29,No,3,0
2,Nuevo,1,1,F,NaN,NaN,NaN,LIMA,LIMA,JESUS MARIA,...,UTP Lima Centro,Fac. Der. Cienc. Polit. Y RRII,DERECHO,NOCHE,SIN BENEFICIO,Presencial,4. 24-29,No,1,0
3,Nuevo,0,0,F,COLEGIO,1220 SAN JOSE MARELLO,ESTATAL,LIMA,LIMA,ATE,...,UTP Lima Centro,Fac. Adm. Y Neg.,ADMINISTRACION DE NEGOCIOS (50/50),NOCHE,SIN BENEFICIO,Presencial,4. 24-29,No,0,0
4,Nuevo,1,1,F,INSTITUTO,IDAT,PRIVADA,LIMA,LIMA,SURQUILLO,...,UTP Lima Centro,Fac. Adm. Y Neg.,ADM. DE NEGOCIOS INTERNACIO,MAÑANA,CONVENIOS,Presencial,4. 24-29,No,2,0


In [87]:
df.describe()

,TUITION PAYMENT MARCH 2022,TUITION PAYMENT MARCH 2023,NUMBER OF ENROLLED COURSES,AT-RISK COURSE
count,37582.000000,37582.000000,37582.000000,37582.000000
mean,0.861476,0.841334,1.940078,0.148422
std,0.345453,0.365370,0.986984,0.512012
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,0.000000
50%,1.000000,1.000000,2.000000,0.000000
75%,1.000000,1.000000,3.000000,0.000000
max,1.000000,1.000000,6.000000,5.000000


In [88]:
def checknulls():
    for col in df.columns:
        if(df[col].hasnans):
            print(col, df[col].isnull().sum())
checknulls()

GENDER 2
TYPE OF EDUCATIONAL INSTITUTION 21714
EDUCATIONAL INSTITUTION 19370
INSTITUTION STATUS 21714
DEPARTMENT 736
PROVINCE 736
DISTRICT 736
CLASSIFICATION 1
FACULTY 1
PROGRAM/MAJOR 1
SHIFT/SCHEDULE 58
AGE RANGE OF ENROLLED STUDENT 4


In [89]:
df.dropna(subset = ['CLASSIFICATION', 'FACULTY', 'PROGRAM/MAJOR','GENDER', 'AGE RANGE OF ENROLLED STUDENT', 'DEPARTMENT', 'PROVINCE', 'DISTRICT', 'SHIFT/SCHEDULE'], inplace = True)
df['CLASSIFICATION'].hasnans

False

In [90]:
checknulls()

TYPE OF EDUCATIONAL INSTITUTION 21074
EDUCATIONAL INSTITUTION 18756
INSTITUTION STATUS 21074


In [91]:
df['EDUCATIONAL INSTITUTION'][df['TYPE OF EDUCATIONAL INSTITUTION'].isnull()]

2                                   NaN
6                                   NaN
10                          UNIVERSIDAD
11            SACOOLIVEROS-SANTAISABELC
14       CAMARA DE COMERCIO DE BELLAVIS
                      ...              
37576                               NaN
37577                       UNIVERSIDAD
37578                               NaN
37579                               NaN
37581                               NaN
Name: EDUCATIONAL INSTITUTION, Length: 21074, dtype: object

In [92]:
df.columns

Index(['ENROLLMENT', 'TUITION PAYMENT MARCH 2022',
       'TUITION PAYMENT MARCH 2023', 'GENDER',
       'TYPE OF EDUCATIONAL INSTITUTION', 'EDUCATIONAL INSTITUTION',
       'INSTITUTION STATUS', 'DEPARTMENT', 'PROVINCE', 'DISTRICT',
       'CLASSIFICATION', 'CAMPUS', 'FACULTY', 'PROGRAM/MAJOR',
       'SHIFT/SCHEDULE', 'BENEFIT DISCOUNTS', 'STUDY MODE',
       'AGE RANGE OF ENROLLED STUDENT', 'DISABILITY',
       'NUMBER OF ENROLLED COURSES', 'AT-RISK COURSE'],
      dtype='object')

In [93]:
df.groupby(['TYPE OF EDUCATIONAL INSTITUTION', 'INSTITUTION STATUS', 'PROVINCE', 'DISTRICT', 'CAMPUS'])['EDUCATIONAL INSTITUTION'].count()

TYPE OF EDUCATIONAL INSTITUTION  INSTITUTION STATUS  PROVINCE       DISTRICT            CAMPUS         
COLEGIO                          ESTATAL             ABANCAY        ABANCAY             UTP Lima Centro    2
                                                                                        UTP Virtual        4
                                                     ALTO AMAZONAS  LAGUNAS             UTP Chiclayo       1
                                                                    YURIMAGUAS          UTP Lima Centro    1
                                                                                        UTP Virtual        1
                                                                                                          ..
UNIVERSIDAD                      PRIVADA             SANTA          CHIMBOTE            UTP Virtual        1
                                                     TACNA          ALTO DE LA ALIANZA  UTP Arequipa       1
                        

In [94]:
cat_cols = set(df.columns) - {'TYPE OF EDUCATIONAL INSTITUTION', 'EDUCATIONAL INSTITUTION', 'INSTITUTION STATUS'}
for col in df.columns:
    if(not(df[col].dtype==object)):
        cat_cols = cat_cols - {col}
        print(col,len(df[col].unique()),df[col].dtype)
    # if(len(df[col].unique())>10):
    #     print(col,len(df[col].unique()),df[col].dtype)
cat_cols = cat_cols - {"GENDER"}
cat_cols = list(cat_cols)
cat_cols

TUITION PAYMENT MARCH 2022 2 int64
TUITION PAYMENT MARCH 2023 2 int64
NUMBER OF ENROLLED COURSES 7 int64
AT-RISK COURSE 6 int64


['CLASSIFICATION',
 'ENROLLMENT',
 'CAMPUS',
 'DISABILITY',
 'DEPARTMENT',
 'DISTRICT',
 'PROVINCE',
 'PROGRAM/MAJOR',
 'BENEFIT DISCOUNTS',
 'SHIFT/SCHEDULE',
 'FACULTY',
 'AGE RANGE OF ENROLLED STUDENT',
 'STUDY MODE']

In [95]:
for col in cat_cols:
    encoder = LabelEncoder()
    df[col] = encoder.fit_transform(df[col])
df['GENDER'] = df['GENDER'].replace({'M':1, 'F':2, '2':2, '1':1, 'U':3})
df

C:\Users\visha\AppData\Local\Temp\ipykernel_24636\2489943517.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['GENDER'] = df['GENDER'].replace({'M':1, 'F':2, '2':2, '1':1, 'U':3})


,ENROLLMENT,TUITION PAYMENT MARCH 2022,TUITION PAYMENT MARCH 2023,GENDER,TYPE OF EDUCATIONAL INSTITUTION,EDUCATIONAL INSTITUTION,INSTITUTION STATUS,DEPARTMENT,PROVINCE,DISTRICT,...,CAMPUS,FACULTY,PROGRAM/MAJOR,SHIFT/SCHEDULE,BENEFIT DISCOUNTS,STUDY MODE,AGE RANGE OF ENROLLED STUDENT,DISABILITY,NUMBER OF ENROLLED COURSES,AT-RISK COURSE
0,0,0,0,1,INSTITUTO,IDAT,PRIVADA,14,98,54,...,6,6,36,1,4,0,4,0,0,0
1,0,1,0,1,COLEGIO,COLEGIO SISE,PRIVADA,14,98,660,...,6,6,40,2,4,0,3,0,3,0
2,0,1,1,2,NaN,NaN,NaN,14,98,268,...,6,3,24,2,4,0,3,0,1,0
3,0,0,0,2,COLEGIO,1220 SAN JOSE MARELLO,ESTATAL,14,98,32,...,6,0,12,2,4,0,3,0,0,0
4,0,1,1,2,INSTITUTO,IDAT,PRIVADA,14,98,606,...,6,0,4,0,2,0,3,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37577,2,1,1,2,NaN,UNIVERSIDAD,NaN,3,8,95,...,13,4,82,1,4,2,4,0,2,0
37578,2,1,1,2,NaN,NaN,NaN,14,98,324,...,13,4,82,1,4,2,4,0,3,0
37579,2,1,1,2,NaN,NaN,NaN,14,98,308,...,13,0,3,2,4,2,4,0,3,0
37580,2,1,1,1,UNIVERSIDAD,PONTIFICIAUNIVERSIDADCATÓLIC,PRIVADA,14,98,137,...,13,5,55,1,4,2,4,0,3,0


In [97]:
X = df.drop(['TUITION PAYMENT MARCH 2023','TYPE OF EDUCATIONAL INSTITUTION', 'EDUCATIONAL INSTITUTION', 'INSTITUTION STATUS'], axis=1)
y = df['TUITION PAYMENT MARCH 2023']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)